In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report
from pyod.models.iforest import IForest
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, precision_score, recall_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import OneSidedSelection
import os
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from pyod.models.pca import PCA
from sklearn.mixture import GaussianMixture
from sklearn.covariance import EllipticEnvelope
from sklearn.model_selection import GridSearchCV

import os, sys, pickle
import argparse, sys

import seaborn as sns
sns.set_style('white')

import warnings
warnings.filterwarnings('ignore')



# parser = argparse.ArgumentParser()
# parser.add_argument('-label', help=' : Please set the label') 
# args = parser.parse_args()


df = pd.read_csv('data/modified_dataset.csv')


# def remove_outliers(df, column_name, factor=1.5):
#     Q1 = df[column_name].quantile(0.25)
#     Q3 = df[column_name].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - factor * IQR
#     upper_bound = Q3 + factor * IQR
#     df_filtered = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]
#     return df_filtered

# # 모든 칼럼에 대해 이상치 제거 실행
# for column_name in df.columns:
#     df = remove_outliers(df, column_name)







df_stroke_0 = df[df['Cancer'] == 0]
df_stroke_1 = df[df['Cancer'] == 1]



#train, test data processing 1,2,3,4중 선택

#1.정상으로만 학습, 테스트는 정상과 이상 반반
answer_label = 'Cancer'
X = df[df.columns.difference([answer_label])]
df_normal = df[df[answer_label] == 0]
df_abnormal = df[df[answer_label] == 1]
test_normal_df = df_normal.sample(n=len(df_stroke_1), random_state = 0)
test_df = pd.concat([df_abnormal, test_normal_df])
X_test = test_df[test_df.columns.difference([answer_label])]
y_test = test_df[answer_label]
train_df = df_normal.drop(test_normal_df.index)
X_train = train_df[train_df.columns.difference([answer_label])]
y_train = train_df[answer_label]



In [8]:
# Local Outlier Factor

def main_task(contamination, n_neighbors, leaf_size):
    LOF = LocalOutlierFactor(novelty=True, contamination=contamination, n_neighbors=n_neighbors, leaf_size=leaf_size)
    LOF.fit(X_train)
    LOF_test_pred = LOF.predict(X_test) 
    LOF_test_pred = pd.DataFrame(LOF_test_pred)
    LOF_test_pred = LOF_test_pred.replace({-1: 1, 1: 0})
    return round(f1_score(y_test, LOF_test_pred),3)

In [9]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

def optuna_main(trial: optuna.Trial):
    contamination = trial.suggest_float('contamination', 0.01, 0.5, step = 0.02)
    n_neighbors = trial.suggest_int('n_neighbors', 10, 40, step = 1)
    leaf_size = trial.suggest_int('leaf_size', 30, 60, step = 1)
    print('contamination : ', contamination)
    f1_score = main_task(contamination=contamination, n_neighbors=n_neighbors, leaf_size=leaf_size)

    print('<f1_score:', f1_score,'>')
    return f1_score

In [10]:
study = optuna.create_study(direction='maximize')
study.optimize(optuna_main, n_trials=100)
print(f'study.best_trial.value : {study.best_trial.value}')
print(f'study.best_params : {study.best_trial.params}')

[I 2023-11-08 15:11:38,737] A new study created in memory with name: no-name-b4a8a35c-1586-49f7-a00c-5c8a3e0c28f1


contamination :  0.17


[I 2023-11-08 15:11:38,978] Trial 0 finished with value: 0.753 and parameters: {'contamination': 0.17, 'n_neighbors': 10, 'leaf_size': 57}. Best is trial 0 with value: 0.753.


<f1_score: 0.753 >
contamination :  0.37


[I 2023-11-08 15:11:39,276] Trial 1 finished with value: 0.757 and parameters: {'contamination': 0.37, 'n_neighbors': 24, 'leaf_size': 49}. Best is trial 1 with value: 0.757.


...

<f1_score: 0.757 >
contamination :  0.31


[I 2023-11-08 15:12:11,148] Trial 98 finished with value: 0.76 and parameters: {'contamination': 0.31, 'n_neighbors': 26, 'leaf_size': 38}. Best is trial 77 with value: 0.773.


<f1_score: 0.76 >
contamination :  0.35000000000000003


[I 2023-11-08 15:12:11,453] Trial 99 finished with value: 0.757 and parameters: {'contamination': 0.35000000000000003, 'n_neighbors': 24, 'leaf_size': 52}. Best is trial 77 with value: 0.773.


<f1_score: 0.757 >
study.best_trial.value : 0.773
study.best_params : {'contamination': 0.31, 'n_neighbors': 23, 'leaf_size': 30}


In [9]:
#결과
print(f'study.best_trial.value : {study.best_trial.value}')
print(f'study.best_paramse : {study.best_params}')

study.best_trial.value : 0.77
study.best_paramse : {'contamination': 0.29000000000000004}
